In [1]:
import camelot
import json
import os
import re

In [2]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [16]:
hello = get_biggest_table("D:/study/LammaIndex/documents/test1.pdf",50)

{
  "metadata": {
    "source_file": "test1",
    "page": 1,
    "table_order": 1,
    "total_rows": 16,
    "total_columns": 3
  },
  "headers": [
    "STT",
    "hang_hoa",
    "yeu_cau_ky_thuat"
  ],
  "data": [
    {
      "row_index": 1,
      "values": {
        "STT": "IV",
        "hang_hoa": "Bộ chuyển đổi \nnguồn AC/DC \nloại nhỏ (1U)",
        "yeu_cau_ky_thuat": "-  Các loại thiết bị, vật tư, phụ kiện \nphải có nguồn gốc xuất xứ rõ ràng, \ncó  chứng  nhận  chất \nlượng  sản \nphẩm của nhà sản xuất. \n-  Thiết bị mới 100% chưa qua sử \ndụng \n-  Thiết  bị  phải  được  sản  xuất  từ \nnăm 2021 trở lại đây \n-  Thuộc \nloại \nthiết  bị  nguồn  sử \ndụng kỹ thuật chuyển mạch, thiết \nkế  dạng  Module  theo  chức  năng \ncủa từng khối. \n-  Tất cả các khối đặt trong tủ liên \nhoàn, đồng bộ của hãng, tạo thành \nthiết \nbị \ncấp \nnguồn \nvà \nnạp \nacquyhoàn chỉnh. \n-  Thời  gian  bảo  hành:  theo  tiêu \nchuẩn  của  nhà  sản  xuất,  tối  thiểu \n12 tháng."
      }
    },
    {

In [17]:
data = hello["data"]

In [18]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [19]:
converted_data

[{'ten_san_pham': 'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)',
  'cac_muc': [{'ten_hang_hoa': 'Yêu cầu chung', 'thong_so_ky_thuat': {}},
   {'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
    'thong_so_ky_thuat': {'94C74': 'Kích thước: < 2U',
     '48499': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3',
     '34DE8': 'Công suất mỗi module chỉnh lưu: ≥ 1000W',
     '976D6': 'Số lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 module',
     'CA0B1': 'Attomat DC: + Attomat cho tải: ●  Loại 30A: ≥ 02 cái ●  Loại 10A: ≥ 01 cái ●  Loại 03A: ≥ 01 cái + Attomat cho acquy: ●  Loại 50A: ≥ 01 cái'}},
   {'ten_hang_hoa': 'Đầu vào AC',
    'thong_so_ky_thuat': {'AE067': 'Sử dụng được các điện áp L＋N ＋PE/220VAC',
     '181CA': 'Dải điện áp đầu vào từ 85VAC ÷ 300VAC',
     '2A970': 'Dải tần số AC đầu vào: từ 45Hz ÷ 65Hz.'}},
   {'ten_hang_hoa': 'Đầu ra DC',
    'thong_so_ky_thuat': {'8B9FD': 'Điện  áp  đầu ra  danh  định - 48VDC, cực dương đấu đất',
     'CA203': 'Dải điện áp đầu ra: từ -43.2VDC tới

In [20]:
context_queries = {}  # Dict chứa thông tin chi tiết theo key
product_key = {}  # Dict lồng: ten_san_pham -> ten_hang_hoa -> list[key]

for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                q = value[0]
                k = value[1]
                value_str = ' '.join(value)
            else:
                q = None
                k = value
                value_str = value

            # Ghi vào context_queries
            context_queries[key] = {
                "ten_san_pham": ten_san_pham,
                "ten_hang_hoa": ten_hang_hoa,
                "value": value_str,
                "yeu_cau_ky_thuat_chi_tiet": k,
                "yeu_cau_ky_thuat": q
            }

            # Ghi vào product_key
            if ten_san_pham not in product_key:
                product_key[ten_san_pham] = {}
            if ten_hang_hoa not in product_key[ten_san_pham]:
                product_key[ten_san_pham][ten_hang_hoa] = []
            product_key[ten_san_pham][ten_hang_hoa].append(key)



In [21]:
context_queries

{'94C74': {'ten_san_pham': 'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Kích thước: < 2U',
  'yeu_cau_ky_thuat_chi_tiet': 'Kích thước: < 2U',
  'yeu_cau_ky_thuat': None},
 '48499': {'ten_san_pham': 'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3',
  'yeu_cau_ky_thuat': None},
 '34DE8': {'ten_san_pham': 'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Công suất mỗi module chỉnh lưu: ≥ 1000W',
  'yeu_cau_ky_thuat_chi_tiet': 'Công suất mỗi module chỉnh lưu: ≥ 1000W',
  'yeu_cau_ky_thuat': None},
 '976D6': {'ten_san_pham': 'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 module',
  'yeu_cau_ky_thuat

In [22]:
product_key

{'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)': {'Cấu hình thiết bị nguồn': ['94C74',
   '48499',
   '34DE8',
   '976D6',
   'CA0B1'],
  'Đầu vào AC': ['AE067', '181CA', '2A970'],
  'Đầu ra DC': ['8B9FD', 'CA203', 'ACAE0', '79EA6', '958BA', '3C2CD'],
  'Yêu cầu với module chỉnh lưu (Rectifier)': ['80076',
   '618FD',
   'DCB9E',
   '4827E',
   '5A21D',
   'BDFB1',
   '35BC0',
   'D3686'],
  'Tính năng của thiết bị nguồn': ['9E7AD',
   'BBAB8',
   '73ADB',
   '1C0EE',
   '59689',
   '49F9E',
   'B2F1A'],
  'Khối điều khiển và hiển thị': ['76F42', 'AAA98', 'D0049', '0581D', '2795C'],
  'Điều kiện làm việc': ['DCED8'],
  'Hệ thống làm mát': ['0D04B'],
  'Điều kiện bảo hành': ['37398']}}

In [23]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
import time
load_dotenv()

clientOpenAi = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [24]:
def retrieve_product_line(product_name, assistant_id="asst_CkhqaSBGeaIlLO5mY7puPybD"):
    thread = clientOpenAi.beta.threads.create()
    thread_id = thread.id
    # 2. Gửi message vào thread
    clientOpenAi.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=product_name
    )
    run = clientOpenAi.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice="auto"  # hoặc thay bằng tool cụ thể nếu cần
        # tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )
    run_id = run.id
    # 4. Đợi assistant xử lý xong
    while True:
        run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled", "expired"]:
            raise Exception(f"Run failed with status: {run_status.status}")
        time.sleep(1)

    # 5. Lấy kết quả trả về từ Assistant
    messages = clientOpenAi.beta.threads.messages.list(thread_id=thread_id)
    for message in reversed(messages.data):  # đảo ngược để lấy kết quả mới nhất trước
        if message.role == "assistant":
            for content in message.content:
                if content.type == "text":
                    return content.text.value

    return None

In [25]:
def create_prompt_extract_module(query_str):
    prompt = f"""
You are an expert in hardware product documentation analysis.  
Read the provided text (which can be either a detailed product brochure or a general product requirement) and extract ONLY the core physical hardware components/modules of the system.
 
For each component:
- If the text explicitly includes a model number, code, or exact specification tied to the component → output "<Full Component Name>: <Exact Model(s)/Code(s)>".
- If the text does NOT provide a model number or code → output only "<Full Component Name>".
 
Input:
<<<
{query_str}
>>>
 
Output format:
- <Component Name>[: <Model(s)/Code(s) if available>]
 
Rules:
1. Only include core hardware modules essential for the product’s operation (e.g., Rectifier Module, Controller, AC Input, AC Distribution, DC Distribution, Battery Distribution, Lightning Protection, Cooling System, Battery Bank).
2. Preserve the exact wording of component/module names from the text (do not paraphrase or generalize).
3. Include model numbers, codes, or exact designations only if explicitly stated in the text.  
   - If multiple models exist, list them separated by " / ".
4. If a component has sub-parts (e.g., BLVD/LLVD, Input/Output), keep them as separate lines with their full names.
5. Ignore optional accessories, warranty info, standards compliance, and marketing text unless they are part of the official component name/specification.
7. Do not infer or guess component names—extract only what is explicitly stated.    
"""
    return prompt

In [26]:
def create_prompt_extract_module2(query_str):
    prompt = f"""
    You are an expert in hardware product documentation analysis.  
Read the provided text (which can be either a detailed product brochure or a general product requirement) and extract ONLY the model numbers, codes, or exact designations of the core physical hardware components/modules of the system.

Input:
<<<
{query_str}
>>>

Output format:
A valid JSON array of strings, where each string is one model/code.  
Example:
["R48-121A3", "R56-3220"]

Rules:
1. Only extract model numbers, codes, or exact designations explicitly stated in the text.  
   - Do NOT include component/module names, descriptions, amperage, voltage, or units (e.g., "125 A / 2P" is ignored).
2. Extract models only from core hardware modules essential for the product’s operation (e.g., Rectifier Module, Controller, AC Input, AC Distribution, DC Distribution, Battery Distribution, Lightning Protection, Cooling System, Battery Bank).
3. Preserve the exact case, spacing, and characters from the original text.
4. If multiple models are listed together, split them into separate JSON array elements.
5. Ignore optional accessories, warranty info, standards compliance, and marketing text.
6. Do not infer or guess model numbers—extract only what is explicitly stated.
7. Output only a valid JSON array without extra text or explanations.

    """
    return prompt

In [27]:
product = 'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U)'
product_line = retrieve_product_line(product)
print(f"Product Line: {product_line}")
query_str = f""

all_requirements = product_key[product]
for key in all_requirements:
    query_str += f"{key} :"
    for item in all_requirements[key]:
        if item not in context_queries:
            continue
        query_str += context_queries[item]["value"]
    query_str += "\n"
    print(query_str)
prompt_yeu_cau_ky_thuat = create_prompt_extract_module(query_str)
response = clientOpenAi.responses.create(
    model="gpt-4o-mini",
    input=prompt_yeu_cau_ky_thuat,
    temperature=0
)
product_requirement = f"{product}: {response.output_text.strip()}" 


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2492\258005419.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = clientOpenAi.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2492\258005419.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  clientOpenAi.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2492\258005419.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAi.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2492\258005419.py:19: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2492\258005419.py:27: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = clientOpenAi.be

Product Line: "DC Power Systems"
Cấu hình thiết bị nguồn :Kích thước: < 2USố lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3Công suất mỗi module chỉnh lưu: ≥ 1000WSố lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 moduleAttomat DC: + Attomat cho tải: ●  Loại 30A: ≥ 02 cái ●  Loại 10A: ≥ 01 cái ●  Loại 03A: ≥ 01 cái + Attomat cho acquy: ●  Loại 50A: ≥ 01 cái

Cấu hình thiết bị nguồn :Kích thước: < 2USố lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3Công suất mỗi module chỉnh lưu: ≥ 1000WSố lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 moduleAttomat DC: + Attomat cho tải: ●  Loại 30A: ≥ 02 cái ●  Loại 10A: ≥ 01 cái ●  Loại 03A: ≥ 01 cái + Attomat cho acquy: ●  Loại 50A: ≥ 01 cái
Đầu vào AC :Sử dụng được các điện áp L＋N ＋PE/220VACDải điện áp đầu vào từ 85VAC ÷ 300VACDải tần số AC đầu vào: từ 45Hz ÷ 65Hz.

Cấu hình thiết bị nguồn :Kích thước: < 2USố lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3Công suất mỗi module chỉnh lưu: ≥ 1000WSố lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥

In [28]:
product_requirement

'Bộ chuyển đổi nguồn AC/DC loại nhỏ (1U): - Kích thước\n- Số lượng khe cắm module chỉnh lưu (Rectifier)\n- Công suất mỗi module chỉnh lưu\n- Số lượng module chỉnh lưu trang bị kèm tủ nguồn\n- Attomat DC\n- Attomat cho tải\n- Attomat cho acquy\n- Đầu vào AC\n- Dải điện áp đầu vào\n- Dải tần số AC đầu vào\n- Đầu ra DC\n- Điện áp đầu ra danh định\n- Dải điện áp đầu ra\n- Dòng tải max\n- Độ ổn định điện áp đầu ra\n- Nhiễu băng rộng (Wide band noise)\n- Độ gợn sóng đầu ra (đỉnh – đỉnh)\n- Yêu cầu với module chỉnh lưu (Rectifier)\n- Hệ số công suất của Rectifier\n- Hiệu suất đỉnh module rectifier\n- Tổng độ méo hài THD của Rectifier\n- Rectifier thay thế nóng\n- Trạng thái hoạt động của Rectifier\n- Tính năng của thiết bị nguồn\n- Bộ nguồn\n- Hệ thống có trang bị chức năng LLVD (Load Low Voltage Disconection)\n- Khối điều khiển và hiển thị\n- Màn hình LCD\n- Bảng điều khiển\n- Giao diện cho cổng giao tiếp\n- Hệ thống làm mát\n- Làm mát bằng quạt'

In [45]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)
import time


# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="hello_my_friend",
    client=client,
    aclient=aclient,
)

def retrieve_document(product_line, query_str):
    product_ids = []
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_document = MetadataFilters(
        filters=[
            MetadataFilter(key="product_line", operator=FilterOperator.EQ, value=product_line),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="summary_document"),
        ],
    condition=FilterCondition.AND,
    )
    retriever_document = index.as_retriever(similarity_top_k=5, sparse_top_k=10, verbose=True, enable_hybrid=True, filters=filters_document,alpha=0.7)
    
    results = retriever_document.retrieve(query_str)
    # print("product: ", results)
    # print("results: ", results)
    for result in results:
        metadata = result.metadata
        print(metadata)
        product_ids.append(
            {
                "product_id": metadata["product_id"],
                "brochure_file_path": metadata["brochure_file_path"],
            }
        )

    return product_ids

async def retrieve_chunk_async(product_ids, query_str):
    """
    Phiên bản async của retrieve_chunk
    """
    # Wrap các operations đồng bộ trong executor để không block event loop
    loop = asyncio.get_event_loop()
    
    def _retrieve_sync():
        index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
        
        filters_chunk = MetadataFilters(
            filters=[
                MetadataFilter(key="product_id", operator=FilterOperator.IN, value=product_ids),
                MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
            ],
            condition=FilterCondition.AND,
        )
        retriever_chunk = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_chunk)
        
        results = retriever_chunk.retrieve(query_str)
        content = ""
        for i, result in enumerate(results, start=1):
            metadata = result.metadata
            file_name = metadata["file_name"] + ".pdf"
            page = metadata["page"]
            table = metadata["table_name"]
            figure_name = metadata.get("figure_name")
            text = result.text.strip()
            content += f"Chunk {i} trong file {file_name} tại trang {page}, có chứa bảng {table} và hình {figure_name} có nội dung:\n{text}\n\n"
        return content
    
    # Chạy function đồng bộ trong thread pool để không block
    return await loop.run_in_executor(None, _retrieve_sync)

def retrieve_product_line(product_name, assistant_id="asst_j5wHMN84dpSLXD2GMH5QifS0"):
    from openai import OpenAI 
    clientOpenAi = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    thread = clientOpenAi.beta.threads.create()
    thread_id = thread.id
    # 2. Gửi message vào thread
    clientOpenAi.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=product_name
    )
    run = clientOpenAi.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice="auto"  # hoặc thay bằng tool cụ thể nếu cần
        # tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )
    run_id = run.id
    # 4. Đợi assistant xử lý xong
    while True:
        run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled", "expired"]:
            raise Exception(f"Run failed with status: {run_status.status}")
        time.sleep(1)

    # 5. Lấy kết quả trả về từ Assistant
    messages = clientOpenAi.beta.threads.messages.list(thread_id=thread_id)
    for message in reversed(messages.data):  # đảo ngược để lấy kết quả mới nhất trước
        if message.role == "assistant":
            for content in message.content:
                if content.type == "text":
                    # print("product_line: ", content.text.value)
                    return content.text.value

    return None

def retrieve_component(keyword_list):
    # Tạo danh sách điều kiện OR
    should_conditions = [
        FieldCondition(
            key='file_brochure_name',
            match=MatchText(text=kw)
        )
        for kw in keyword_list
    ]

    text_filter = Filter(
        should=should_conditions  # OR search
    )

    scroll_result, next_page = client.scroll(
        collection_name="hello_my_friend",
        scroll_filter=text_filter,
        limit=5
    )

    product_ids = []
    if scroll_result:
        print("Kết quả tìm kiếm:")
        for result in scroll_result:
            metadata = result.payload
            product_id = metadata.get("product_id", "")
            if product_id:
                product_ids.append(product_id)
    print(product_ids)
    return product_ids


        




In [46]:
product_line = product_line.replace('"', '').replace("'", "")
print(product_line)  # DC Power Systems

DC Power Systems


In [47]:
products = retrieve_document(product_line, product_requirement)

{'category': 'Critical Power', 'product_line': 'DC Power Systems', 'product_name': 'Vertiv™ Mini NetSure™ Control Unit', 'summary': 'A compact, pluggable DC power system controller designed for dense power environments, enabling remote monitoring and control of rectifiers, converters, batteries, and site conditions. It manages the full DC power chain.Integrated control of AC mains, rectifiers, batteries, and environment. Remote management via web (HTTPS), SNMP, Modbus, TL1. Advanced battery management: thermal protection, reserve time prediction. Hot-pluggable and supports multilingual interfaces. Expandable via optional interface boards for more I/Os.', 'product_id': '7866b3f6-78bd-11f0-928b-38f3abb08dd1', 'type': 'summary_document', 'brochure_file_path': 'output/M831A Controller Datasheet.md', 'file_brochure_name': 'M831A Controller Datasheet.pdf'}
{'category': 'Critical Power', 'product_line': 'DC Power Systems', 'product_name': '\u200bNetSure 701', 'summary': 'The NetSure™ 701 seri

In [48]:
products

[{'product_id': '7866b3f6-78bd-11f0-928b-38f3abb08dd1',
  'brochure_file_path': 'output/M831A Controller Datasheet.md'},
 {'product_id': '5ac3396c-78c0-11f0-a69c-38f3abb08dd1',
  'brochure_file_path': 'output/netsure-701-series_datasheet_en-asia.md'},
 {'product_id': '967d4216-78c2-11f0-8aad-38f3abb08dd1',
  'brochure_file_path': 'output/esure-rectifier-1r483000e3b-data-sheet.md'},
 {'product_id': '6f65920b-78c0-11f0-8850-38f3abb08dd1',
  'brochure_file_path': 'output/netsure-801-brochure.md'},
 {'product_id': 'e3b19345-78bc-11f0-85a4-38f3abb08dd1',
  'brochure_file_path': 'output/netsure-2100-brochure.md'}]

In [68]:
item = products[4]

In [69]:
item

{'product_id': 'c9ab50bc-78bb-11f0-86ca-38f3abb08dd1',
 'brochure_file_path': 'output/NetSure -731 A41 Brochure.md'}

In [70]:
product_search_id = []
product_id = item["product_id"]
brochure = item["brochure_file_path"]

In [71]:
from llama_index.readers.file import MarkdownReader
import json

# Khởi tạo reader
reader = MarkdownReader()
documents = reader.load_data(file=f"D:/study/LammaIndex/{brochure}")
markdown_text = "\n".join(doc.text for doc in documents)

In [72]:
prompt_brochure = create_prompt_extract_module2(markdown_text)
response = clientOpenAi.responses.create(
    model="gpt-4o-mini",
    input=prompt_brochure,
    temperature=0
)
product_brochure = response.output_text.strip()

In [73]:
import json
import re
# Cách 1: Dùng regex để lấy phần bên trong code block
match = re.search(r'```json\s*(.*?)\s*```', product_brochure, re.DOTALL)
if match:
    json_str = match.group(1)
else:
    json_str = response  # nếu không có code block thì dùng nguyên văn

# Parse JSON thành list Python
product_brochure = json.loads(json_str)

In [86]:
product_brochure

['NetSure™ 731 A41',
 'R48-3000A3',
 'R48-3000e3',
 'R48-3500e3',
 'R48-3500E4',
 'M221S',
 'M830B']

In [75]:
from qdrant_client.http.models import PayloadSchemaType, Filter, FieldCondition, MatchText
product_component_id = retrieve_component(product_brochure)

Kết quả tìm kiếm:
['a78fd7ff-78bb-11f0-a0c0-38f3abb08dd1', '58875529-78bc-11f0-9d8f-38f3abb08dd1', 'c9ab50bc-78bb-11f0-86ca-38f3abb08dd1']


In [76]:
product_search_id=[]
product_search_id.extend(product_component_id) 
product_search_id.append(product_id)

In [77]:
product_search_id = set(product_search_id)

In [84]:
product_search_id

{'58875529-78bc-11f0-9d8f-38f3abb08dd1',
 'a78fd7ff-78bb-11f0-a0c0-38f3abb08dd1',
 'c9ab50bc-78bb-11f0-86ca-38f3abb08dd1'}

In [ ]:
context_queries

In [88]:
kha_nang_dap_ung_tham_chieu_final = {}
kha_nang_dap_ung_tham_chieu_step = {}
async def process_single_item(item: str, key: str, context_queries: Dict, product_search_id: Any) -> tuple:
    """
    Xử lý một item đơn lẻ một cách bất đồng bộ
    """
    if item not in context_queries:
        return item, None
    
    query = context_queries[item]["value"]
    content = await retrieve_chunk_async(product_search_id, query)
    print(f"Processed item: {item}")  # In ra để theo dõi tiến trình
    
    return item, {
        'relevant_context': content
    }
# Phiên bản với giới hạn số lượng concurrent tasks (khuyến nghị)
async def process_requirements_async_with_semaphore(all_requirements: Dict, context_queries: Dict, product_search_id: Any, max_concurrent: int = 10) -> Dict:
    """
    Phiên bản async với giới hạn số lượng concurrent tasks
    """
    kha_nang_dap_ung_tham_chieu_step = {}
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def process_with_semaphore(item: str, key: str):
        async with semaphore:
            return await process_single_item(item, key, context_queries, product_search_id)
    
    # Tạo danh sách tất cả các tasks với semaphore
    tasks = []
    for key in all_requirements:
        for item in all_requirements[key]:
            task = process_with_semaphore(item, key)
            tasks.append(task)
    
    print(f"Bắt đầu xử lý {len(tasks)} tasks với tối đa {max_concurrent} concurrent...")
    
    # Chạy tất cả tasks đồng thời (nhưng giới hạn concurrent)
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Xử lý kết quả
    for result in results:
        if isinstance(result, Exception):
            print(f"Lỗi khi xử lý: {result}")
            continue
        
        item, data = result
        if data is not None:
            kha_nang_dap_ung_tham_chieu_step[item] = data
    
    return kha_nang_dap_ung_tham_chieu_step
kha_nang_dap_ung_tham_chieu_step = await process_requirements_async_with_semaphore(
        all_requirements, context_queries, product_search_id, max_concurrent=5
    )

Bắt đầu xử lý 45 tasks với tối đa 5 concurrent...
Processed item: B6B13
Processed item: A13CF
Processed item: 53F0F
Processed item: 6266E
Processed item: EF540
Processed item: 48E25
Processed item: 79BBE
Processed item: 46045
Processed item: 9C791
Processed item: 53211
Processed item: FCB4C
Processed item: 3D45D
Processed item: EC8D4
Processed item: 660E9
Processed item: E3A93
Processed item: 602C6
Processed item: FDD3C
Processed item: FFE72
Processed item: 7C3EA
Processed item: AFECD
Processed item: 76B1A
Processed item: 5F427
Processed item: 6E617
Processed item: 3F0D7
Processed item: FC55E
Processed item: 31B97
Processed item: 1D174
Processed item: 13DCF
Processed item: 95399
Processed item: 981D1
Processed item: F23C4
Processed item: 5EF9F
Processed item: 94585
Processed item: 6C105
Processed item: FF2ED
Processed item: 336B2
Processed item: 158E4
Processed item: C60F3
Processed item: DD0A8
Processed item: E5B19
Processed item: 8949D
Processed item: A2B89
Processed item: 9A88B
Proc

In [89]:
kha_nang_dap_ung_tham_chieu_step

{'6266E': {'relevant_context': 'Chunk 1 trong file Netsure 731 A41 Usermanual.pdf tại trang 30, có chứa bảng Technical Data of Power System và hình None có nội dung:\n## Appendix 1 Technical And Engineering Data\n\n| Table 1 Technical data<br/>Parameter category | Table 1 Technical data<br/>Parameter             | Table 1 Technical data<br/>Description                                                                                          |\n| --------------------------------------------- | ------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------- |\n| Environmental                                 | Operating temperature                            | －10℃～＋70℃(derating is necessary above 40°C)                                                                                     |\n|                                               | Storage temperature                    

In [41]:
from openai import OpenAI
import json 
import time
import asyncio
from typing import Dict, Any

clientOpenAI = OpenAI()


DEFAULT_OBJECT = {
    "yeu_cau_ky_thuat": "",
    "kha_nang_dap_ung": "",
    "tai_lieu_tham_chieu": {
        "file": "",
        "section": "",
        "table_or_figure": "",
        "page": 0,
        "evidence": ""
    }
}

def fill_defaults(data: dict, defaults: dict) -> dict:
    """
    Đệ quy bổ sung các trường mặc định vào data nếu thiếu.
    """
    result = defaults.copy()
    for key, default_value in defaults.items():
        if key in data:
            if isinstance(default_value, dict) and isinstance(data[key], dict):
                result[key] = fill_defaults(data[key], default_value)
            else:
                result[key] = data[key]
    return result


def extract_first_json_object(json_str: str):
    s = json_str.strip()
    
    # Tìm dấu '{' đầu tiên
    start_index = s.find('{')
    if start_index == -1:
        print("❌ Không tìm thấy JSON object nào.")
        return DEFAULT_OBJECT

    # Duyệt từ đó để tìm dấu '}' kết thúc object đầu tiên
    brace_count = 0
    end_index = -1
    for i in range(start_index, len(s)):
        if s[i] == '{':
            brace_count += 1
        elif s[i] == '}':
            brace_count -= 1
            if brace_count == 0:
                end_index = i + 1  # Cắt đến sau dấu '}'
                break
    if end_index == -1:
        print("❌ Không tìm thấy JSON đóng đúng.")
        return DEFAULT_OBJECT

    first_json_str = s[start_index:end_index]
    # Kiểm tra xem có parse được không
    try:
        result = json.loads(first_json_str)
        # Bổ sung field mặc định nếu thiếu
        return fill_defaults(result, DEFAULT_OBJECT)
    except json.JSONDecodeError:
        return DEFAULT_OBJECT

# === ASYNC VERSION OF TRACK_REFERENCE ===
async def track_reference_async(context_queries: Dict, kha_nang_dap_ung_tham_chieu_step: Dict, 
                               max_concurrent: int = 5) -> Dict:
    """
    Phiên bản async của track_reference với giới hạn số requests đồng thời
    """
    assistant_id = "asst_FZIBIfjPM3kCoxURARvM27UV"
    
    # Tạo semaphore để giới hạn số requests đồng thời
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def process_single_item(key: str):
        async with semaphore:  # Giới hạn số requests đồng thời
            try:
                value = context_queries[key]["value"]
                content = kha_nang_dap_ung_tham_chieu_step[key]["relevant_context"]
                
                user_prompt = f"""
                Chunk và metadata: {content}
                Yêu cầu: {value}
                """
                
                print(f"🚀 Đang xử lý key: {key}")
                result = await evaluate_technical_requirement_async(user_prompt, assistant_id)
                
                if isinstance(result, str):
                    result = extract_first_json_object(result)
                
                return key, result
                
            except Exception as e:
                print(f"❌ Lỗi xử lý key {key}: {str(e)}")
                return key, DEFAULT_OBJECT
    
    # Tạo tasks cho tất cả items
    tasks = [process_single_item(key) for key in kha_nang_dap_ung_tham_chieu_step]
    
    # Chạy tất cả tasks song song
    print(f"🏃‍♂️ Bắt đầu xử lý {len(tasks)} items với {max_concurrent} requests đồng thời...")
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Xử lý kết quả
    for result in results:
        if isinstance(result, Exception):
            print(f"❌ Task failed: {result}")
            continue
            
        key, processed_result = result
        
        kha_nang_dap_ung_tham_chieu_step[key]["kha_nang_dap_ung"] = processed_result.get("kha_nang_dap_ung", "")
        kha_nang_dap_ung_tham_chieu_step[key]["tai_lieu_tham_chieu"] = {
            "file": processed_result["tai_lieu_tham_chieu"]["file"],
            "section": processed_result["tai_lieu_tham_chieu"].get("section", ""),
            "table_or_figure": processed_result["tai_lieu_tham_chieu"].get("table_or_figure", ""),
            "page": processed_result["tai_lieu_tham_chieu"].get("page", 0),
            "evidence": processed_result["tai_lieu_tham_chieu"].get("evidence", "")
        }
        kha_nang_dap_ung_tham_chieu_step[key].pop("relevant_context", None)
        print(f"✅ Hoàn thành key: {key}")
    
    print("🎉 Hoàn thành tất cả!")
    return kha_nang_dap_ung_tham_chieu_step
# === ASYNC VERSION CHO ĐOẠN CODE CỦA BẠN ===
async def process_requirements_async(context_queries: Dict, kha_nang_dap_ung_tham_chieu_step: Dict,
                                   max_concurrent: int = 5):
    """
    Phiên bản async cho đoạn code gốc của bạn
    """
    assistant_id = "asst_FZIBIfjPM3kCoxURARvM27UV"
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def process_item(key: str):
        async with semaphore:
            try:
                value = context_queries[key]["value"]
                content = kha_nang_dap_ung_tham_chieu_step[key]["relevant_context"]
                form = context_queries[key]["yeu_cau_ky_thuat_chi_tiet"]
                
                user_prompt = f'''
                Chunk và metadata: {content}
                Yêu cầu: {value}
                '''
                
                print(f"🚀 Processing: {key}")
                
                # Gọi hàm đánh giá (async version)
                result = await evaluate_technical_requirement_async(user_prompt, assistant_id)
                
                if isinstance(result, str):
                    result = extract_first_json_object(result)
                
                return key, result
                
            except Exception as e:
                print(f"❌ Error processing {key}: {e}")
                return key, DEFAULT_OBJECT
    
    # Tạo và chạy tất cả tasks
    tasks = [process_item(key) for key in kha_nang_dap_ung_tham_chieu_step]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Cập nhật kết quả
    for result in results:
        if isinstance(result, Exception):
            continue
            
        key, processed_result = result
        
        kha_nang_dap_ung_tham_chieu_step[key]["kha_nang_dap_ung"] = processed_result.get('kha_nang_dap_ung', "")
        kha_nang_dap_ung_tham_chieu_step[key]["tai_lieu_tham_chieu"] = {
            "file": processed_result['tai_lieu_tham_chieu']['file'],
            "section": processed_result['tai_lieu_tham_chieu'].get('section', ''),
            "table_or_figure": processed_result['tai_lieu_tham_chieu'].get('table_or_figure', ''),
            "page": processed_result['tai_lieu_tham_chieu'].get('page', 0),
            "evidence": processed_result['tai_lieu_tham_chieu'].get('evidence', '')
        }
        kha_nang_dap_ung_tham_chieu_step[key].pop("relevant_context", None)
        
        print(f"✅ Completed: {key}")

# Hàm tạo thread
def create_thread():
    thread = clientOpenAI.beta.threads.create()
    return thread.id

# === UPDATE ASSISTANT ===
def update_assistant(assistant_id):
    assistant = clientOpenAI.beta.assistants.update(
        assistant_id=assistant_id,
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === EVALUATE TECHNICAL REQUIREMENT ===
async def evaluate_technical_requirement_async(user_prompt: str, assistant_id: str) -> Dict[str, Any]:
    """
    Phiên bản async của evaluate_technical_requirement
    """
    def _sync_evaluate():
        # 1. Tạo thread riêng cho mỗi lần gọi
        thread = clientOpenAI.beta.threads.create()
        thread_id = thread.id

        # 2. Gửi message vào thread
        clientOpenAI.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_prompt
        )

        # 3. Tạo run
        run = clientOpenAI.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id,
            tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
        )

        # 4. Chờ assistant xử lý (tối đa 20s)
        for _ in range(20):
            run = clientOpenAI.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            if run.status not in ["queued", "in_progress"]:
                break
            time.sleep(1)

        # 5. Lấy arguments trực tiếp
        if run.status == "requires_action":
            call = run.required_action.submit_tool_outputs.tool_calls[0]
            print(f"👉 Assistant đã gọi tool: {call.function.name}")
            print("🧠 Dữ liệu JSON assistant muốn trả về:")
            print(call.function.arguments)
            return call.function.arguments

        elif run.status == "completed":
            messages = clientOpenAI.beta.threads.messages.list(thread_id=thread_id)
            for msg in messages.data:
                print(f"[{msg.role}] {msg.content[0].text.value}")
            return None

        else:
            print(f"Run status: {run.status}")
            return None
    
    # Chạy function sync trong thread pool
    return await asyncio.to_thread(_sync_evaluate)


In [42]:
# === CÁCH SỬ DỤNG ===

# Thay thế đoạn code gốc của bạn bằng:
await process_requirements_async(
    context_queries, 
    kha_nang_dap_ung_tham_chieu_step,
    max_concurrent=10  # Số requests đồng thời (tùy chỉnh)
)

🚀 Processing: 59F0F
🚀 Processing: 41A92
🚀 Processing: B180C
🚀 Processing: 42BC5
🚀 Processing: 7C60F
🚀 Processing: 2AF36
🚀 Processing: 3AA33
🚀 Processing: 19BE0
🚀 Processing: A2C97
🚀 Processing: 9636B


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\674877805.py:211: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = clientOpenAI.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\674877805.py:215: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  clientOpenAI.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\674877805.py:222: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAI.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\674877805.py:230: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAI.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)


👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Điện áp đầu ra danh định -48VDC, cực dương đấu đất","kha_nang_dap_ung":"\"\"","tai_lieu_tham_chieu":{"file":"M831A Controller User Manual.pdf","section":"","table_or_figure":"","page":0,"evidence":""}}
🚀 Processing: E804B
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Sử dụng được các điện áp L＋N＋PE/220VAC","kha_nang_dap_ung":"","tai_lieu_tham_chieu":{"file":"M831A Controller User Manual.pdf","section":"","table_or_figure":"","page":0,"evidence":""}}
🚀 Processing: 0CB16
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Dải điện áp đầu ra: từ -43.2VDC tới -57.6VDC","kha_nang_dap_ung":"","tai_lieu_tham_chieu":{"file":"M831A Controller User Manual.pdf","section":"","table_or_figure":"","page":0,"evidence":""}}
🚀 Processing: F4A82
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu J

In [ ]:
kha_nang_dap_ung_tham_chieu_step

In [44]:
from typing import Dict, Any, Tuple
# === ASYNC VERSION OF ADAPT_OR_NOT ===
async def adapt_or_not_async(kha_nang_dap_ung_tham_chieu_step: Dict, 
                           adapt_or_not_step: Dict, 
                           all_requirements: Dict,
                           context_queries: Dict,
                           max_concurrent: int = 5) -> Tuple[Dict, Dict]:
    """
    Phiên bản async của hàm adapt_or_not
    """
    assistant_id = "asst_SIWbRtRbvCxXS9dgqvtj9U8O"
    print(f"Assistant ID: {assistant_id}")
    
    # Tạo semaphore để giới hạn số requests đồng thời
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def process_requirement(key: str):
        async with semaphore:
            try:
                print(f"🚀 Đang xử lý requirement: {key}")
                
                dap_ung_ky_thuat = ""
                tai_lieu_tham_chieu = ""
                
                # Thu thập thông tin từ tất cả items trong requirement
                for item in all_requirements[key]:
                    if item not in kha_nang_dap_ung_tham_chieu_step:
                        continue
                        
                    yeu_cau_ky_thuat = context_queries[item].get('yeu_cau_ky_thuat_chi_tiet', "")
                    kha_nang_dap_ung = kha_nang_dap_ung_tham_chieu_step[item].get('kha_nang_dap_ung', "")
                    dap_ung_ky_thuat += f"{yeu_cau_ky_thuat} || {kha_nang_dap_ung}\n"
            
                    tai_lieu = kha_nang_dap_ung_tham_chieu_step[item].get('tai_lieu_tham_chieu', {})
                    file = tai_lieu.get("file", "")
                    page = tai_lieu.get("page", "")
                    table_or_figure = tai_lieu.get("table_or_figure", "")
                    evidence = tai_lieu.get("evidence", "")
            
                    tai_lieu_text = f"{file}, trang: {page}"
                    if table_or_figure:
                        tai_lieu_text += f", trong bảng(figure): {table_or_figure}"
                    tai_lieu_text += f", evidence: {evidence}\n\n"
                    tai_lieu_tham_chieu += tai_lieu_text
                
                # Chỉ xử lý nếu có dữ liệu
                if dap_ung_ky_thuat and tai_lieu_tham_chieu:
                    print(f"📞 Gọi API cho key: {key}")
                    result = await Evaluator_adaptability_async(dap_ung_ky_thuat, assistant_id)
                    result = parse_output_text(result)  # result đã là dict
                    
                    output_text = result['đáp ứng kỹ thuật']
                    
                    print(f"✅ Hoàn thành key: {key}")
                    return key, output_text, tai_lieu_tham_chieu
                else:
                    print(f"⚠️ Không có dữ liệu cho key: {key}")
                    return key, None, None
                    
            except Exception as e:
                print(f"❌ Lỗi xử lý key {key}: {str(e)}")
                return key, None, None
    
    # Tạo tasks cho tất cả requirements
    tasks = [process_requirement(key) for key in all_requirements]
    
    print(f"🏃‍♂️ Bắt đầu xử lý {len(tasks)} requirements với {max_concurrent} requests đồng thời...")
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Xử lý kết quả
    for result in results:
        if isinstance(result, Exception):
            print(f"❌ Task failed: {result}")
            continue
            
        key, output_text, tai_lieu_tham_chieu = result
        
        if output_text is not None and tai_lieu_tham_chieu is not None:
            if key not in adapt_or_not_step:
                adapt_or_not_step[key] = []
            
            adapt_or_not_step[key].append(output_text)
            adapt_or_not_step[key].append(tai_lieu_tham_chieu)
    
    print("🎉 Hoàn thành tất cả requirements!")
    return kha_nang_dap_ung_tham_chieu_step, adapt_or_not_step



def parse_output_text(output_text: str) -> dict:
    DEFAULT_JSON = {"đáp ứng kỹ thuật": "0"}
    # B1: Tìm phần JSON đầu tiên trong chuỗi
    match = re.search(r"\{.*\}", output_text, re.DOTALL)
    if not match:
        return DEFAULT_JSON.copy()

    json_str = match.group(0).strip()

    # B2: Parse JSON
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        return DEFAULT_JSON.copy()

    # B3: Nếu không có key thì trả mặc định
    if "đáp ứng kỹ thuật" not in data:
        return DEFAULT_JSON.copy()

    return data



# Hàm tạo thread
def create_thread():
    thread = clientOpenAI.beta.threads.create()
    return thread.id

# === ASYNC VERSION OF EVALUATOR_ADAPTABILITY ===
async def Evaluator_adaptability_async(user_prompt: str, assistant_id: str = "asst_SIWbRtRbvCxXS9dgqvtj9U8O") -> str:
    """
    Phiên bản async của Evaluator_adaptability
    """
    def _sync_evaluate():
        # 1. Tạo thread riêng cho mỗi lần gọi
        thread = clientOpenAI.beta.threads.create()
        thread_id = thread.id

        # 2. Gửi message vào thread
        clientOpenAI.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_prompt
        )

        # 3. Tạo run
        run = clientOpenAI.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id,
            tool_choice={"type": "function", "function": {"name": "evaluator"}}
        )

        # 4. Chờ assistant xử lý (tối đa 20s)
        for _ in range(20):
            run = clientOpenAI.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            if run.status not in ["queued", "in_progress"]:
                break
            time.sleep(1)

        # 5. Lấy arguments trực tiếp
        if run.status == "requires_action":
            call = run.required_action.submit_tool_outputs.tool_calls[0]
            print(f"👉 Assistant đã gọi tool: {call.function.name}")
            print("🧠 Dữ liệu JSON assistant muốn trả về:")
            print(call.function.arguments)
            return call.function.arguments

        elif run.status == "completed":
            messages = clientOpenAI.beta.threads.messages.list(thread_id=thread_id)
            for msg in messages.data:
                print(f"hello:.........[{msg.role}] {msg.content[0].text.value}")
            return None

        else:
            print(f"Run status: {run.status}")
            return None
    
    # Chạy function sync trong thread pool
    return await asyncio.to_thread(_sync_evaluate)

In [45]:
adapt_or_not_step = {}
adapt_or_not_final = {}

In [46]:
kha_nang_dap_ung_tham_chieu_step, adapt_or_not_step = await adapt_or_not_async(
    kha_nang_dap_ung_tham_chieu_step,
    adapt_or_not_step,
    all_requirements,  # Cần pass thêm biến này
    context_queries,   # Cần pass thêm biến này
    max_concurrent=10   # Số requests đồng thời
)

Assistant ID: asst_SIWbRtRbvCxXS9dgqvtj9U8O
🏃‍♂️ Bắt đầu xử lý 9 requirements với 10 requests đồng thời...
🚀 Đang xử lý requirement: Đầu vào AC
📞 Gọi API cho key: Đầu vào AC
🚀 Đang xử lý requirement: Đầu ra DC
📞 Gọi API cho key: Đầu ra DC
🚀 Đang xử lý requirement: Yêu cầu với module chỉnh lưu (Rectifier)
📞 Gọi API cho key: Yêu cầu với module chỉnh lưu (Rectifier)
🚀 Đang xử lý requirement: Tính năng của thiết bị nguồn
📞 Gọi API cho key: Tính năng của thiết bị nguồn
🚀 Đang xử lý requirement: Khối điều khiển và hiển thị
📞 Gọi API cho key: Khối điều khiển và hiển thị
🚀 Đang xử lý requirement: Điều kiện làm việc
📞 Gọi API cho key: Điều kiện làm việc
🚀 Đang xử lý requirement: Hệ thống làm mát
📞 Gọi API cho key: Hệ thống làm mát
🚀 Đang xử lý requirement: Điều kiện bảo hành
📞 Gọi API cho key: Điều kiện bảo hành
🚀 Đang xử lý requirement: Acquy kèm theo
📞 Gọi API cho key: Acquy kèm theo


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\3849892291.py:125: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = clientOpenAI.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\3849892291.py:129: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  clientOpenAI.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\3849892291.py:136: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAI.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8056\3849892291.py:144: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAI.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)


👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"1/2"}
✅ Hoàn thành key: Điều kiện làm việc
👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"0/5"}
✅ Hoàn thành key: Đầu ra DC
👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"2/12"}
✅ Hoàn thành key: Acquy kèm theo
👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"0"}
✅ Hoàn thành key: Yêu cầu với module chỉnh lưu (Rectifier)
👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"0"}
✅ Hoàn thành key: Hệ thống làm mát
👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"1"}
✅ Hoàn thành key: Điều kiện bảo hành
👉 Assistant đã gọi tool: evaluator
🧠 Dữ liệu JSON assistant muốn trả về:
{"đáp ứng kỹ thuật":"1/7"}
✅ Hoàn thành key: Khối điều khiển và hiển thị
👉 Assistant đã gọi tool: evalu

In [ ]:
kha_nang_dap_ung_tham_chieu_step

In [ ]:
adapt_or_not_step

In [132]:
def merge_dicts(kha_nang_dap_ung_tham_chieu_step, context_queries):
    for k, v in kha_nang_dap_ung_tham_chieu_step.items():
        if k in context_queries and isinstance(v, dict) and isinstance(context_queries[k], dict):
            # Nếu cả 2 cùng là dict thì merge đệ quy
            merge_dicts(v, context_queries[k])
        else:
            # Nếu không phải dict hoặc key chưa tồn tại trong B thì gán trực tiếp
            context_queries[k] = v
    return context_queries

In [133]:
context_queries= merge_dicts(kha_nang_dap_ung_tham_chieu_step, context_queries)

In [ ]:
context_queries

In [145]:
for key in adapt_or_not_step:
    product_key['Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)'][key].extend(adapt_or_not_step[key]) 

In [ ]:
product_key

In [ ]:
sửa prompt để llm chỉ bóc ra được những component được chỉ định(phụ thuộc website)